In [19]:
import os
import joblib
import numpy as np
import pandas as pd
import mlflow
import boto3
import mlflow.sklearn
from pathlib import Path
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
from mlflow.exceptions import MlflowException
from dotenv import load_dotenv
from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score, recall_score,
    confusion_matrix, log_loss
)

In [20]:
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

tracking_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"
registry_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"

client = mlflow.MlflowClient(tracking_uri=tracking_uri, registry_uri=registry_uri)

In [29]:
EXPERIMENT_NAME = os.getenv('EXPERIMENT_NAME')

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
print(experiment_id)

548588041841417201


In [48]:
experiment_runs = mlflow.search_runs(
    experiment_ids=[experiment_id],
).sort_values(by="start_time", ascending=False) 
print(experiment_runs)

                             run_id       experiment_id    status   
0  2b7d61e40b8c479eb5ce180e9cc02bb6  548588041841417201  FINISHED  \
1  5c29f724b94847dd9a1c4f3b5707aa20  548588041841417201  FINISHED   
2  fef253a81e2142fea7b68d93889eff66  548588041841417201  FINISHED   
3  75e96ee5b12e4bf79816d83f65f2eea7  548588041841417201  FINISHED   
4  a7800744c6be4c6480c47c19c60a5715  548588041841417201  FINISHED   
5  47aa14ef623e45a490a3d5236fcbaa75  548588041841417201  FINISHED   
6  c1538cae1c984f1189696003c6956fc1  548588041841417201    FAILED   
7  4ac72c692a0449abb091901853bde3f9  548588041841417201  FINISHED   

                                        artifact_uri   
0  file:///home/mle-user/mle_projects/mle-mlflow/...  \
1  file:///home/mle-user/mle_projects/mle-mlflow/...   
2  file:///home/mle-user/mle_projects/mle-mlflow/...   
3  file:///home/mle-user/mle_projects/mle-mlflow/...   
4  file:///home/mle-user/mle_projects/mle-mlflow/...   
5  file:///home/mle-user/mle_projects/mle-

In [23]:
experiment_runs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   run_id                           8 non-null      object             
 1   experiment_id                    8 non-null      object             
 2   status                           8 non-null      object             
 3   artifact_uri                     8 non-null      object             
 4   start_time                       8 non-null      datetime64[ns, UTC]
 5   end_time                         8 non-null      datetime64[ns, UTC]
 6   metrics.total_charges_mean       8 non-null      float64            
 7   metrics.end_date_nan             8 non-null      float64            
 8   metrics.monthly_charges_median   8 non-null      float64            
 9   metrics.monthly_charges_max      8 non-null      float64            
 10  metric

In [24]:
runs = experiment_runs[[
    "run_id", 
    "start_time", 
    "metrics.err1", 
    "metrics.err2", 
    "metrics.logloss", 
    "metrics.recall", 
    "metrics.auc", 
    "metrics.f1", 
    "metrics.precision"
]]

KeyError: "['metrics.err1', 'metrics.err2', 'metrics.logloss', 'metrics.recall', 'metrics.auc', 'metrics.f1', 'metrics.precision'] not in index"

In [27]:
# фиксируем интересующий запуск (run_id) и метрику
run_id = "90e7f8d96c234294a6cd521d6d356331"
metric = "logloss"

print(client.get_metric_history(run_id, metric))

[<Metric: key='logloss', step=0, timestamp=1756651249525, value=0.15596488537618183>]


In [ ]:
#[<Metric: key='logloss', step=0, timestamp=1697722260419, value=7.3417519678329475>]

In [28]:
client.get_run(run_id).data.metrics

{'auc': 0.9687619453769603,
 'precision': 0.9565217391304348,
 'recall': 0.8257372654155496,
 'f1': 0.8863309352517986,
 'err1': 14.0,
 'err2': 65.0,
 'err1_rate': 0.009971509971509971,
 'err2_rate': 0.046296296296296294,
 'logloss': 0.15596488537618183}

In [ ]:
# {'err1': 0.057487579843860895,
# 'err2': 0.1242015613910575,
# 'auc': 0.8255639980800066,
# 'precision': 0.68359375,
# 'recall': 0.45931758530183725,
# 'f1': 0.5494505494505495,
# 'logloss': 7.3417519678329475} 

In [31]:
run = client.get_run(run_id)
artifact_uri = run.info.artifact_uri

print(f"Путь артефакта в S3: '{artifact_uri}'")

Путь артефакта в S3: 's3://s3-student-mle-20250802-8a8f1a1def-freetrack/5/90e7f8d96c234294a6cd521d6d356331/artifacts'


In [32]:
mlflow.artifacts.download_artifacts(artifact_uri, dst_path="./artifacts_local")

'/home/mle-user/mle_projects/mle-mlflow/artifacts_local/artifacts'

In [47]:
REGISTRY_MODEL_NAME = "churn_model_vadimshakula_b2c"  # или ..._b2c, если подтвердилось
REGISTRY_MODEL_VERSION = "1"

model_uri = f"models:/{REGISTRY_MODEL_NAME}/{REGISTRY_MODEL_VERSION}"
model = mlflow.pyfunc.load_model(model_uri)        # работает для любого flavor
# preds = model.predict(X_test)  # пример инференса

MlflowException: Registered Model with name=churn_model_vadimshakula_b2c not found